# Оценка эффективности новой фичи
## Задачи:
### 1. Посчитать показатели Retention Rate седьмого дня и ARPPU за два периода: до и после внедрения фичи.
### 2. Сформировать выводы по анализу.
* Примечание: Период до внедрения фичи: май. Период после внедрения фичи: июнь

In [19]:
# Импортируем библиотека
import pandas as pd
import numpy as np
import psycopg2
import psycopg2.extras
import warnings
warnings.filterwarnings("ignore")

### 1. Счиатем показатели за май
#### Получим данные за май, найдём Retention Rate и ARPU

In [20]:
# Получаем данные о пользователях, зарегистрировнных в мае, сохраняем в ДатаФрейм
def getRegisteredUsersCountMay():
    query = '''SELECT r.registration_date, count(distinct(r.user_id)) as registered_users_count
    FROM case9.registration r
    WHERE r.registration_date >= '2019-05-01' and r.registration_date < '2019-06-01'
    GROUP BY 1
    '''.format()
    conn = psycopg2.connect("dbname='skillfactory' user='skillfactory' host='84.201.134.129' password='cCkxxLVrDE8EbvjueeMedPKt' port=5432")
    dict_cur = conn.cursor(cursor_factory=psycopg2.extras.DictCursor)
    dict_cur.execute(query)
    rows = dict_cur.fetchall()
    data = []
    for row in rows:
        data.append(dict(row))
    return data
registered_users_count_may = pd.DataFrame(getRegisteredUsersCountMay())
# Меняем формат на Datetime
registered_users_count_may['registration_date'] = pd.to_datetime(registered_users_count_may['registration_date'])
registered_users_count_may.head(10)

,registration_date,registered_users_count
0,2019-05-01,6516
1,2019-05-02,5309
2,2019-05-03,6266
3,2019-05-04,8639
4,2019-05-05,9787
5,2019-05-06,7561
6,2019-05-07,6199
7,2019-05-08,6650
8,2019-05-09,5634
9,2019-05-10,5247


In [21]:
# Получаем данные о пользователях, зарегистрировнных в мае, и их активности, сохраняем в ДатаФрейм
def getActiveUsersCountWithCohortsMay():
    query = '''SELECT ac.activity_date, r.registration_date, count(distinct(ac.user_id)) as active_users_count
    FROM case9.user_activity ac
    LEFT JOIN case9.registration r ON r.user_id = ac.user_id
    WHERE r.registration_date >= '2019-05-01' and r.registration_date < '2019-06-01'
    GROUP BY 1,2
    '''.format()
    conn = psycopg2.connect("dbname='skillfactory' user='skillfactory' host='84.201.134.129' password='cCkxxLVrDE8EbvjueeMedPKt' port=5432")
    dict_cur = conn.cursor(cursor_factory=psycopg2.extras.DictCursor)
    dict_cur.execute(query)
    rows = dict_cur.fetchall()
    data = []
    for row in rows:
        data.append(dict(row))
    return data
active_users_count_with_cohorts_may = pd.DataFrame(getActiveUsersCountWithCohortsMay())
# Меняем формат на Datetime
for col in ['activity_date', 'registration_date']:
    active_users_count_with_cohorts_may[col] = pd.to_datetime(active_users_count_with_cohorts_may[col])
active_users_count_with_cohorts_may.head(10)


,activity_date,registration_date,active_users_count
0,2019-05-01,2019-05-01,2590
1,2019-05-02,2019-05-01,2146
2,2019-05-02,2019-05-02,2049
3,2019-05-03,2019-05-01,1637
4,2019-05-03,2019-05-02,1817
5,2019-05-03,2019-05-03,2389
6,2019-05-04,2019-05-01,1451
7,2019-05-04,2019-05-02,1350
8,2019-05-04,2019-05-03,2126
9,2019-05-04,2019-05-04,3323


In [22]:
# Объединяем ДатаФреймы по дате регистрации
retention_table_may = active_users_count_with_cohorts_may.merge(registered_users_count_may,
                                                                on=['registration_date'], how='left')
retention_table_may.head(10)

,activity_date,registration_date,active_users_count,registered_users_count
0,2019-05-01,2019-05-01,2590,6516
1,2019-05-02,2019-05-01,2146,6516
2,2019-05-02,2019-05-02,2049,5309
3,2019-05-03,2019-05-01,1637,6516
4,2019-05-03,2019-05-02,1817,5309
5,2019-05-03,2019-05-03,2389,6266
6,2019-05-04,2019-05-01,1451,6516
7,2019-05-04,2019-05-02,1350,5309
8,2019-05-04,2019-05-03,2126,6266
9,2019-05-04,2019-05-04,3323,8639


In [23]:
# Получим  параметр lifetime и будем использовать данные только 7-го дня активности, потом посчитаем Retention Rate
# Определим количество дней с даты регистрации до даты активности
retention_table_may['lifetime'] = retention_table_may['activity_date'] - retention_table_may['registration_date']
# Приведём количество дней к формату число
retention_table_may['lifetime'] = retention_table_may['lifetime']/np.timedelta64(1, 'D')
retention_table_may['lifetime'] = retention_table_may['lifetime'].astype(int)
retention_table_may.head(10)
# Так как задача стоит сравнить только 7-й день активности, сохраним в
# отдельный датафрейм только данные об активности на 7 день
retention_table_may_7 = retention_table_may[retention_table_may['lifetime'] == 7]

In [24]:
# Считаем Retention Rate, найдём отношение количества активных пользователей active_users_count к количеству
# зарегистрировавшихся пользователей registered_users_count в этой когорте.
retention_table_may_7['retention_rate'] = retention_table_may_7['active_users_count'] / retention_table_may_7['registered_users_count']
retention_table_may_7.head()
# Считаем и сохраняем в переменную итоговый Retention Rate когорты мая на 7 день
retention_rate_may_7 = round(retention_table_may_7['active_users_count'].sum() / retention_table_may_7['registered_users_count'].sum(), 4)
print('Retantion Rate до внедрения фичи для пользователей на 7-й день активности: {}%'.format(retention_rate_may_7*100))

Retantion Rate до внедрения фичи для пользователей на 7-й день активности: 12.93%


In [25]:
# Получаем данные о  выручке по дням в разрезе когорт для пользователей, зарегистрировавшихся в мае 2019 года.
# После чего получим данные по запросу и запишем в датафрейм revenue.
def getRevenueMay():
    query = '''SELECT re.date, r.registration_date, count(distinct(re.user_id)) as users_count_with_revenue, sum(re.revenue) as revenue
    FROM case9.user_revenue re
    LEFT JOIN case9.registration r ON r.user_id = re.user_id
    WHERE r.registration_date >= '2019-05-01' and r.registration_date < '2019-06-01'
    GROUP BY 1,2
    '''.format()
    conn = psycopg2.connect("dbname='skillfactory' user='skillfactory' host='84.201.134.129' password='cCkxxLVrDE8EbvjueeMedPKt' port=5432")
    dict_cur = conn.cursor(cursor_factory=psycopg2.extras.DictCursor)
    dict_cur.execute(query)
    rows = dict_cur.fetchall()
    data = []
    for row in rows:
        data.append(dict(row))
    return data
revenue_may = pd.DataFrame(getRevenueMay())
# Приводим нужные колонки к формату datetime
for col in ['date', 'registration_date']:
    revenue_may[col] = pd.to_datetime(revenue_may[col])
revenue_may.head(10)

,date,registration_date,users_count_with_revenue,revenue
0,2019-05-01,2019-05-01,1672,2357.590
1,2019-05-02,2019-05-01,1420,514.679
2,2019-05-02,2019-05-02,1367,1629.140
3,2019-05-03,2019-05-01,1048,390.952
4,2019-05-03,2019-05-02,1164,533.608
5,2019-05-03,2019-05-03,1544,1433.580
6,2019-05-04,2019-05-01,931,260.137
7,2019-05-04,2019-05-02,858,433.893
8,2019-05-04,2019-05-03,1373,555.391
9,2019-05-04,2019-05-04,2131,1786.990


In [26]:
# Объединим новый ДФ с revenue_may
# Переименуем колонки в ДФ revenue, чтобы они отличались в объединённом ДФ
revenue_may = revenue_may.rename(columns={'date':'activity_date'})
retention_table_with_revenue_may = retention_table_may.merge(revenue_may,
                                                             on=['registration_date', 'activity_date'], how='left')
# Заменим пустые значения на ноль(дни когда выручки не было)
for col in ['revenue', 'users_count_with_revenue']:
    retention_table_with_revenue_may[col] = retention_table_with_revenue_may[col].fillna(0)
# Количество пользователей с выручкой приведём к типу инт
retention_table_with_revenue_may['users_count_with_revenue'] = retention_table_with_revenue_may['users_count_with_revenue'].astype(int)
# Считаем arpu - делим выручку на кол-во активных пользователей
retention_table_with_revenue_may['arpu'] = \
    retention_table_with_revenue_may['revenue'] / retention_table_with_revenue_may['active_users_count']
retention_table_with_revenue_may.head(10)

,activity_date,registration_date,active_users_count,registered_users_count,lifetime,users_count_with_revenue,revenue,arpu
0,2019-05-01,2019-05-01,2590,6516,0,1672,2357.590,0.910266
1,2019-05-02,2019-05-01,2146,6516,1,1420,514.679,0.239832
2,2019-05-02,2019-05-02,2049,5309,0,1367,1629.140,0.795090
3,2019-05-03,2019-05-01,1637,6516,2,1048,390.952,0.238822
4,2019-05-03,2019-05-02,1817,5309,1,1164,533.608,0.293675
5,2019-05-03,2019-05-03,2389,6266,0,1544,1433.580,0.600075
6,2019-05-04,2019-05-01,1451,6516,3,931,260.137,0.179281
7,2019-05-04,2019-05-02,1350,5309,2,858,433.893,0.321402
8,2019-05-04,2019-05-03,2126,6266,1,1373,555.391,0.261238
9,2019-05-04,2019-05-04,3323,8639,0,2131,1786.990,0.537764


In [27]:
# Считаем итоговый arppu в в мае
arppu_may = round(retention_table_with_revenue_may['revenue'].sum() /
                  retention_table_with_revenue_may['active_users_count'].sum(), 2)
print('ARPPU до внедрения фичи: ', arppu_may)

ARPPU до внедрения фичи:  0.19


### 2. Считаем все те же показатели для когорт июня - после внедрения фичи
#### Получим данные за июнь, найдём Retention Rate и ARPU

In [28]:
# Получаем данные о пользователях, зарегистрировнных в июне, сохраняем в ДатаФрейм
def getRegisteredUsersCountJune():
    query = '''SELECT r.registration_date, count(distinct(r.user_id)) as registered_users_count
    FROM case9.registration r
    WHERE r.registration_date >= '2019-06-01' and r.registration_date < '2019-07-01'
    GROUP BY 1
    '''.format()
    conn = psycopg2.connect("dbname='skillfactory' user='skillfactory' host='84.201.134.129' password='cCkxxLVrDE8EbvjueeMedPKt' port=5432")
    dict_cur = conn.cursor(cursor_factory=psycopg2.extras.DictCursor)
    dict_cur.execute(query)
    rows = dict_cur.fetchall()
    data = []
    for row in rows:
        data.append(dict(row))
    return data
registered_users_count_june = pd.DataFrame(getRegisteredUsersCountJune())
# Меняем формат на Datetime
registered_users_count_june['registration_date'] = pd.to_datetime(registered_users_count_june['registration_date'])
registered_users_count_june.head(10)

,registration_date,registered_users_count
0,2019-06-01,4833
1,2019-06-02,5255
2,2019-06-03,4193
3,2019-06-04,4194
4,2019-06-05,3998
5,2019-06-06,3908
6,2019-06-07,3183
7,2019-06-08,2987
8,2019-06-09,4484
9,2019-06-10,4007


In [29]:
# Получаем данные о пользователях, зарегистрировнных в июне, и их активности, сохраняем в ДатаФрейм
def getActiveUsersCountWithCohortsJune():
    query = '''SELECT ac.activity_date, r.registration_date, count(distinct(ac.user_id)) as active_users_count
    FROM case9.user_activity ac
    LEFT JOIN case9.registration r ON r.user_id = ac.user_id
    WHERE r.registration_date >= '2019-06-01' and r.registration_date < '2019-07-01'
    GROUP BY 1,2
    '''.format()
    conn = psycopg2.connect("dbname='skillfactory' user='skillfactory' host='84.201.134.129' password='cCkxxLVrDE8EbvjueeMedPKt' port=5432")
    dict_cur = conn.cursor(cursor_factory=psycopg2.extras.DictCursor)
    dict_cur.execute(query)
    rows = dict_cur.fetchall()
    data = []
    for row in rows:
        data.append(dict(row))
    return data
active_users_count_with_cohorts_june = pd.DataFrame(getActiveUsersCountWithCohortsJune())
# Меняем формат на Datetime
for col in ['activity_date', 'registration_date']:
    active_users_count_with_cohorts_june[col] = pd.to_datetime(active_users_count_with_cohorts_june[col])
active_users_count_with_cohorts_june.head(10)
# Объединяем ДатаФреймы по дате регистрации
retention_table_june = active_users_count_with_cohorts_june.merge(registered_users_count_june,
                                                                on=['registration_date'], how='left')
retention_table_june.head(10)

,activity_date,registration_date,active_users_count,registered_users_count
0,2019-06-01,2019-06-01,1651,4833
1,2019-06-02,2019-06-01,1429,4833
2,2019-06-02,2019-06-02,1933,5255
3,2019-06-03,2019-06-01,1021,4833
4,2019-06-03,2019-06-02,1550,5255
5,2019-06-03,2019-06-03,1432,4193
6,2019-06-04,2019-06-01,849,4833
7,2019-06-04,2019-06-02,1163,5255
8,2019-06-04,2019-06-03,1244,4193
9,2019-06-04,2019-06-04,1473,4194


In [30]:
# Получим  параметр lifetime и будем использовать данные только 7-го дня активности, потом посчитаем Retention Rate

# Определим количество дней с даты регистрации до даты активности
retention_table_june['lifetime'] = retention_table_june['activity_date'] - retention_table_june['registration_date']
# Приведём количество дней к формату число
retention_table_june['lifetime'] = retention_table_june['lifetime']/np.timedelta64(1, 'D')
retention_table_june['lifetime'] = retention_table_june['lifetime'].astype(int)
retention_table_june.head(10)

,activity_date,registration_date,active_users_count,registered_users_count,lifetime
0,2019-06-01,2019-06-01,1651,4833,0
1,2019-06-02,2019-06-01,1429,4833,1
2,2019-06-02,2019-06-02,1933,5255,0
3,2019-06-03,2019-06-01,1021,4833,2
4,2019-06-03,2019-06-02,1550,5255,1
5,2019-06-03,2019-06-03,1432,4193,0
6,2019-06-04,2019-06-01,849,4833,3
7,2019-06-04,2019-06-02,1163,5255,2
8,2019-06-04,2019-06-03,1244,4193,1
9,2019-06-04,2019-06-04,1473,4194,0


In [31]:
# Так как задача стоит сравнить только 7-й день активности, сохраним в
# отдельный датафрейм только данные об активности на 7 день
retention_table_june_7 = retention_table_june[retention_table_may['lifetime'] == 7]

# Считаем Retention Rate, найдём отношение количества активных пользователей active_users_count к количеству
# зарегистрировавшихся пользователей registered_users_count в этой когорте.
retention_table_june_7['retention_rate'] = \
    retention_table_june_7['active_users_count'] / retention_table_june_7['registered_users_count']
retention_table_june_7.head()

,activity_date,registration_date,active_users_count,registered_users_count,lifetime,retention_rate
28,2019-06-08,2019-06-01,545,4833,7,0.112766
37,2019-06-09,2019-06-02,683,5255,7,0.129971
47,2019-06-10,2019-06-03,484,4193,7,0.115430
58,2019-06-11,2019-06-04,511,4194,7,0.121841
70,2019-06-12,2019-06-05,472,3998,7,0.118059


In [32]:
# Считаем и сохраняем в переменную итоговый Retention Rate когорты мая на 7 день
retention_rate_june_7 = round(retention_table_june_7['active_users_count'].sum() /
                              retention_table_june_7['registered_users_count'].sum(), 4)
print('Retantion Rate после внедрения фичи для пользователей на 7-й '
      'день активности: {}%'.format(retention_rate_june_7*100))

Retantion Rate после внедрения фичи для пользователей на 7-й день активности: 11.87%


In [33]:
# Получаем данные о  выручке по дням в разрезе когорт для пользователей, зарегистрировавшихся в июне 2019 года.
# После чего получим данные по запросу и запишем в датафрейм revenue.
def getRevenueJune():
    query = '''SELECT re.date, r.registration_date, count(distinct(re.user_id)) as users_count_with_revenue, sum(re.revenue) as revenue
    FROM case9.user_revenue re
    LEFT JOIN case9.registration r ON r.user_id = re.user_id
    WHERE r.registration_date >= '2019-06-01' and r.registration_date < '2019-07-01'
    GROUP BY 1,2
    '''.format()
    conn = psycopg2.connect("dbname='skillfactory' user='skillfactory' host='84.201.134.129' password='cCkxxLVrDE8EbvjueeMedPKt' port=5432")
    dict_cur = conn.cursor(cursor_factory=psycopg2.extras.DictCursor)
    dict_cur.execute(query)
    rows = dict_cur.fetchall()
    data = []
    for row in rows:
        data.append(dict(row))
    return data
revenue_june = pd.DataFrame(getRevenueJune())
# Приводим нужные колонки к формату datetime
for col in ['date', 'registration_date']:
    revenue_june[col] = pd.to_datetime(revenue_june[col])
revenue_june.head(10)

,date,registration_date,users_count_with_revenue,revenue
0,2019-06-01,2019-06-01,1082,1611.8600
1,2019-06-02,2019-06-01,928,317.7380
2,2019-06-02,2019-06-02,1257,1305.2200
3,2019-06-03,2019-06-01,623,192.2730
4,2019-06-03,2019-06-02,1008,554.8590
5,2019-06-03,2019-06-03,905,1026.9100
6,2019-06-04,2019-06-01,585,76.8127
7,2019-06-04,2019-06-02,755,248.2080
8,2019-06-04,2019-06-03,802,302.5570
9,2019-06-04,2019-06-04,940,839.7940


In [34]:
# Объединим новый ДФ с revenue_june
# Переименуем колонки в ДФ revenue, чтобы они отличались в объединённом ДФ
revenue_june = revenue_june.rename(columns={'date':'activity_date'})
retention_table_with_revenue_june = retention_table_june.merge(revenue_june,
                                                             on=['registration_date', 'activity_date'], how='left')
# Заменим пустые значения на ноль(дни когда выручки не было)
for col in ['revenue', 'users_count_with_revenue']:
    retention_table_with_revenue_june[col] = retention_table_with_revenue_june[col].fillna(0)
# Количество пользователей с выручкой приведём к типу инт
retention_table_with_revenue_june['users_count_with_revenue'] = \
    retention_table_with_revenue_june['users_count_with_revenue'].astype(int)
# Считаем arpu - делим выручку на кол-во активных пользователей
retention_table_with_revenue_june['arpu'] = \
    retention_table_with_revenue_june['revenue'] / retention_table_with_revenue_june['active_users_count']
retention_table_with_revenue_june.head(10)

,activity_date,registration_date,active_users_count,registered_users_count,lifetime,users_count_with_revenue,revenue,arpu
0,2019-06-01,2019-06-01,1651,4833,0,1082,1611.8600,0.976293
1,2019-06-02,2019-06-01,1429,4833,1,928,317.7380,0.222350
2,2019-06-02,2019-06-02,1933,5255,0,1257,1305.2200,0.675230
3,2019-06-03,2019-06-01,1021,4833,2,623,192.2730,0.188318
4,2019-06-03,2019-06-02,1550,5255,1,1008,554.8590,0.357974
5,2019-06-03,2019-06-03,1432,4193,0,905,1026.9100,0.717116
6,2019-06-04,2019-06-01,849,4833,3,585,76.8127,0.090474
7,2019-06-04,2019-06-02,1163,5255,2,755,248.2080,0.213420
8,2019-06-04,2019-06-03,1244,4193,1,802,302.5570,0.243213
9,2019-06-04,2019-06-04,1473,4194,0,940,839.7940,0.570125


In [35]:
# Считаем итоговый arppu в в июне
arppu_june = round(retention_table_with_revenue_june['revenue'].sum() /
                  retention_table_with_revenue_june['active_users_count'].sum(), 2)
print('ARPPU после внедрения фичи: ', arppu_june)

ARPPU после внедрения фичи:  0.24


In [36]:
# Посчитаем на сколько процентных пунктов изменился retantion rate после внедрения фичи
rr_growth = retention_rate_june_7 - retention_rate_may_7
print('Изменение Retantion Rate после внедрения фичи составило: {} процентных пунктов'.format(round(rr_growth*100, 2)))
# Посчитаем процент роста ARPPU после внедрения фичи
arppu_growth = (arppu_june - arppu_may) / arppu_may
print('Рост средней выручки на платящего пользователя после внедрения фичи составил {}%'.format(round(arppu_growth*100, 2)))

Изменение Retantion Rate после внедрения фичи составило: -1.06 процентных пунктов
Рост средней выручки на платящего пользователя после внедрения фичи составил 26.32%


## Выводы
#### 1. После внедрения фичи коэфициент удержания (Retantion Rate) на 7-й день снизился на 1 процентный пункт. Показатель падения незначительный, для более глубоких выводов о влиянии фичи на показатель требуется более глубокая проработка, в т.ч. анализ RR на 30-й день
#### 2. После внедрения фичи средняя выручка на платящего пользователя (ARPPU) выросла на 26,32 %. Это хороший рост, поэтому фичу можно считать эффективной по этому параметру.